## Importing Libraries

In [4]:
import pandas as pd 
import numpy as np 
import joblib
import re 
import nltk 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

## Uploading Dataset

In [8]:
fake = pd.read_csv('Fake.csv')

In [9]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


## Taking first 5000 rows of data

In [12]:
fake_5000 = fake[:5000]

In [14]:
fake_5000.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [16]:
true = pd.read_csv('True.csv')

In [18]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [20]:
true_5000 = true[:5000]

In [22]:
true_5000.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


## Labeling fake news 0 and true news as 1

In [24]:
fake_5000['label'] = 0
true_5000['label'] = 1

C:\Users\mohammad ehtesham\AppData\Local\Temp\ipykernel_44388\2984978189.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_5000['label'] = 0
C:\Users\mohammad ehtesham\AppData\Local\Temp\ipykernel_44388\2984978189.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_5000['label'] = 1


In [26]:
fake_5000.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [28]:
true_5000.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


## Concating both fake_news and true_news dataset

In [30]:
df = pd.concat([fake_5000, true_5000], axis = 0)

In [32]:
df

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
4995,U.S. Agriculture secretary nominee submits eth...,(Reuters) - U.S. President Donald Trump’s nomi...,politicsNews,"March 13, 2017",1
4996,Trump aides attack agency that will analyze he...,WASHINGTON (Reuters) - Aides to U.S. President...,politicsNews,"March 12, 2017",1
4997,Highlights: The Trump presidency on March 12 a...,(Reuters) - Highlights of the day for U.S. Pre...,politicsNews,"March 12, 2017",1
4998,Obama lawyers move fast to join fight against ...,WASHINGTON (Reuters) - When Johnathan Smith re...,politicsNews,"March 13, 2017",1


## Taking only two columns from the dataset

In [34]:
df = df[['text', 'label']]

In [36]:
df

,text,label
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
...,...,...
4995,(Reuters) - U.S. President Donald Trump’s nomi...,1
4996,WASHINGTON (Reuters) - Aides to U.S. President...,1
4997,(Reuters) - Highlights of the day for U.S. Pre...,1
4998,WASHINGTON (Reuters) - When Johnathan Smith re...,1


## Applying EDA

In [38]:
df['label'].value_counts()

label
0    5000
1    5000
Name: count, dtype: int64

In [40]:
df['text'].isnull().sum()

0

In [42]:
df.duplicated().sum()


38

In [44]:
df = df.drop_duplicates()


In [46]:
df.duplicated().sum()

0

## Applying Text Prepration Steps

In [48]:
df['text'] = df['text'].str.lower()

C:\Users\mohammad ehtesham\AppData\Local\Temp\ipykernel_44388\474617635.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].str.lower()


In [50]:
df

,text,label
0,donald trump just couldn t wish all americans ...,0
1,house intelligence committee chairman devin nu...,0
2,"on friday, it was revealed that former milwauk...",0
3,"on christmas day, donald trump announced that ...",0
4,pope francis used his annual christmas day mes...,0
...,...,...
4995,(reuters) - u.s. president donald trump’s nomi...,1
4996,washington (reuters) - aides to u.s. president...,1
4997,(reuters) - highlights of the day for u.s. pre...,1
4998,washington (reuters) - when johnathan smith re...,1


In [52]:
## Removing all the punctuation

df['text'] = df['text'].apply(lambda x : re.sub(r'[^\w\s]', ' ', x))


C:\Users\mohammad ehtesham\AppData\Local\Temp\ipykernel_44388\998261774.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x : re.sub(r'[^\w\s]', ' ', x))


In [53]:
df

,text,label
0,donald trump just couldn t wish all americans ...,0
1,house intelligence committee chairman devin nu...,0
2,on friday it was revealed that former milwauk...,0
3,on christmas day donald trump announced that ...,0
4,pope francis used his annual christmas day mes...,0
...,...,...
4995,reuters u s president donald trump s nomi...,1
4996,washington reuters aides to u s president...,1
4997,reuters highlights of the day for u s pre...,1
4998,washington reuters when johnathan smith re...,1


In [56]:
df['label'].value_counts()

label
0    5000
1    4962
Name: count, dtype: int64

In [58]:
## Removing all the numbers from dataset

df['text'] = df['text'].apply(lambda x: re.sub(r'\d+', ' ', x))

C:\Users\mohammad ehtesham\AppData\Local\Temp\ipykernel_44388\370910387.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: re.sub(r'\d+', ' ', x))


In [60]:
df

,text,label
0,donald trump just couldn t wish all americans ...,0
1,house intelligence committee chairman devin nu...,0
2,on friday it was revealed that former milwauk...,0
3,on christmas day donald trump announced that ...,0
4,pope francis used his annual christmas day mes...,0
...,...,...
4995,reuters u s president donald trump s nomi...,1
4996,washington reuters aides to u s president...,1
4997,reuters highlights of the day for u s pre...,1
4998,washington reuters when johnathan smith re...,1


## Removing Stopwords

In [62]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\mohammad
[nltk_data]     ehtesham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\mohammad
[nltk_data]     ehtesham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [64]:
stop_words = set(stopwords.words('english'))

In [66]:
def clean_text(text):
    words = word_tokenize(text)
    cleaned_text = [word for word in words if word not in stop_words]
    return " ".join(cleaned_text)

In [68]:
df['text'] = df['text'].apply(clean_text)

C:\Users\mohammad ehtesham\AppData\Local\Temp\ipykernel_44388\1163271480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(clean_text)


In [72]:
df

,text,label
0,donald trump wish americans happy new year lea...,0
1,house intelligence committee chairman devin nu...,0
2,friday revealed former milwaukee sheriff david...,0
3,christmas day donald trump announced would bac...,0
4,pope francis used annual christmas day message...,0
...,...,...
4995,reuters u president donald trump nominee head ...,1
4996,washington reuters aides u president donald tr...,1
4997,reuters highlights day u president donald trum...,1
4998,washington reuters johnathan smith resigned u ...,1


In [74]:
df['text'] = df['text'].apply(lambda x: word_tokenize(x))

C:\Users\mohammad ehtesham\AppData\Local\Temp\ipykernel_44388\1729999017.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(lambda x: word_tokenize(x))


In [76]:
df

,text,label
0,"[donald, trump, wish, americans, happy, new, y...",0
1,"[house, intelligence, committee, chairman, dev...",0
2,"[friday, revealed, former, milwaukee, sheriff,...",0
3,"[christmas, day, donald, trump, announced, wou...",0
4,"[pope, francis, used, annual, christmas, day, ...",0
...,...,...
4995,"[reuters, u, president, donald, trump, nominee...",1
4996,"[washington, reuters, aides, u, president, don...",1
4997,"[reuters, highlights, day, u, president, donal...",1
4998,"[washington, reuters, johnathan, smith, resign...",1


In [78]:
X = df['text'] 
y = df['label']

In [80]:
X.head()

0    [donald, trump, wish, americans, happy, new, y...
1    [house, intelligence, committee, chairman, dev...
2    [friday, revealed, former, milwaukee, sheriff,...
3    [christmas, day, donald, trump, announced, wou...
4    [pope, francis, used, annual, christmas, day, ...
Name: text, dtype: object

In [82]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

In [84]:
X = X.apply(lambda x: " ".join(x))

In [86]:
X

0       donald trump wish americans happy new year lea...
1       house intelligence committee chairman devin nu...
2       friday revealed former milwaukee sheriff david...
3       christmas day donald trump announced would bac...
4       pope francis used annual christmas day message...
                              ...                        
4995    reuters u president donald trump nominee head ...
4996    washington reuters aides u president donald tr...
4997    reuters highlights day u president donald trum...
4998    washington reuters johnathan smith resigned u ...
4999    jakarta reuters u vice president mike pence vi...
Name: text, Length: 9962, dtype: object

## Spliting the dataset into training and testing dataset

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [90]:
X_train

3866    welcome trump america k trumplandia people say...
4071    washington reuters house representatives speak...
2777    washington reuters u senate confirmed private ...
1509    following statements posted verified twitter a...
254     reuters longtime u congressman georgia john le...
                              ...                        
2328    rockville md reuters democratic women packed s...
3782    following statements posted verified twitter a...
3749    donald trump president elect shown sign intent...
2121    washington reuters donald trump thursday fired...
4057    washington reuters conservative house freedom ...
Name: text, Length: 7969, dtype: object

In [92]:
len(X_train)

7969

In [94]:
len(X_test)

1993

## Applying Tfidf

In [96]:
tfidf = TfidfVectorizer(max_features=5000)


In [98]:
X_train_tfidf = tfidf.fit_transform(X_train)

In [99]:
X_test_tfidf = tfidf.transform(X_test)

## Applying GridSeachCv for best model selection

In [102]:
models = {
    "Logistic Regression" : LogisticRegression(),
    "Decision Tree" : DecisionTreeClassifier(),
    "Random Forest" : RandomForestClassifier(),
    "SVM" : SVC(),
    "Naive Bayes" : MultinomialNB()
}

In [104]:
params = {

    "Logistic Regression": {
        "C": [0.1, 1, 10],
        "max_iter": [100, 200]
    },

    "Naive Bayes": {
        "alpha": [0.1, 0.5, 1.0]
    },

    "SVM": {
        "C": [0.1, 1, 10],
        "kernel": ["linear", "rbf"]
    },

    "Decision Tree": {
        "criterion": ["gini", "entropy"],
        "max_depth": [5, 10, 20]
    },

    "Random Forest": {
        "n_estimators": [50, 100],
        "max_depth": [10, 20],
    }
}


In [106]:
scores = []

for name, model in models.items():
    

    clf = GridSearchCV(
        model, 
        params[name], 
        cv=5,
        n_jobs=-1,
        return_train_score=False
    )

    clf.fit(X_train_tfidf, y_train)

    scores.append({
        "model": name,
        "best_score": clf.best_score_,
        "best_params": clf.best_params_
    })


In [108]:
results = pd.DataFrame(scores)
print(results)

                 model  best_score                             best_params
0  Logistic Regression    0.995608              {'C': 10, 'max_iter': 100}
1        Decision Tree    0.998870   {'criterion': 'gini', 'max_depth': 5}
2        Random Forest    0.999121  {'max_depth': 20, 'n_estimators': 100}
3                  SVM    0.997867            {'C': 1, 'kernel': 'linear'}
4          Naive Bayes    0.963859                          {'alpha': 0.1}


In [ ]:
## Applying the model

In [110]:
rf = RandomForestClassifier()

In [112]:
rf.fit(X_train_tfidf, y_train)

RandomForestClassifier()

In [114]:
y_pred = rf.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9979929754139488
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       949
           1       1.00      1.00      1.00      1044

    accuracy                           1.00      1993
   macro avg       1.00      1.00      1.00      1993
weighted avg       1.00      1.00      1.00      1993



In [116]:
joblib.dump(rf, "fake_news_model.pkl")
joblib.dump(tfidf, "tfidf_vectorizer.pkl")

['tfidf_vectorizer.pkl']

In [120]:
X[1]

1    house intelligence committee chairman devin nu...
1    washington reuters transgender people allowed ...
Name: text, dtype: object